In [1]:
import numpy as np
import pandas as pd
import os

In [3]:
hh = pd.read_csv('data/psam_h06.csv', dtype={'RT': 'str', 'SERIALNO': 'str'})
per = pd.read_csv('data/psam_p06.csv',dtype={'RT': 'str', 'SERIALNO': 'str', 'NAICSP': 'str'})

In [4]:
puma_list = [4101,
 4102,
 7701,
 7702,
 7703,
 7704,
 11300,
 6701,
 6702,
 6703,
 6704,
 6705,
 6706,
 6707,
 6708,
 6709,
 6710,
 6711,
 6712,
 4701,
 4702,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 1700,
 9901,
 9902,
 9903,
 9904,
 5301,
 5302,
 5303,
 1301,
 1302,
 1303,
 1304,
 1305,
 1306,
 1307,
 1308,
 9501,
 1309,
 9502,
 9503,
 8501,
 8502,
 8503,
 8504,
 8505,
 8506,
 8507,
 8508,
 8509,
 8510,
 8511,
 8512,
 8513,
 8514,
 7501,
 7502,
 7503,
 7504,
 7505,
 7506,
 7507,
 10100,
 5500,
 8101,
 8102,
 8103,
 8104,
 8105,
 8106,
 6101,
 6102,
 6103,
 9701,
 9702,
 9703,
 8701,
 8702]

print('making copies in memory ...')
hh_update = hh.copy()
per_update = per.copy()


# select puma within the region
hh_update = hh[hh['PUMA'].isin(puma_list)]
per_update = per_update[per_update['PUMA'].isin(puma_list)]

# exclude zero-person household
hh_update= hh_update[hh_update['NP']>0]

making copies in memory ...


In [5]:
per_update['pemploy'] = 999

In [6]:
per_update.loc[(per_update['ESR']==1)|(per_update['ESR']==2)|(per_update['ESR']==4)|(per_update['ESR']==5), 'pemploy'] = 2 # part-time worker
per_update['pemploy'].value_counts()

999    314622
2      294208
Name: pemploy, dtype: int64

In [7]:
per_update.loc[
    ((per_update['ESR']==1)|(per_update['ESR']==2)|(per_update['ESR']==4)|(per_update['ESR']==5))&
    ((per_update['WKW']==1)|(per_update['WKW']==2)|(per_update['WKW']==3)|(per_update['WKW']==4))&
                      (per_update['WKHP']>=35),
                       'pemploy'
                       ] = 1 # full-time worker
per_update['pemploy'].value_counts()

999    314622
1      221073
2       73135
Name: pemploy, dtype: int64

In [10]:
per_update.loc[pd.isna(per_update['ESR']), 'pemploy'] = 4 # student under 16
per_update['pemploy'].value_counts()

1      221073
999    203626
4      110996
2       73135
Name: pemploy, dtype: int64

In [11]:
per_update.loc[(per_update['ESR']==6)|(per_update['ESR']==3), 'pemploy'] = 3  # not in the labor force

In [13]:
per_update['pemploy'].value_counts()

1    221073
3    203626
4    110996
2     73135
Name: pemploy, dtype: int64

In [14]:
per_update['pstudent'] = 999
per_update.loc[(per_update['SCHG']==1)|(per_update['SCHG']==2)|(per_update['SCHG']==3)|(per_update['SCHG']==4)|(per_update['SCHG']==5), 'pstudent'] = 1 # pre-school through grade 12 student
per_update.loc[(per_update['SCHG']==6)|(per_update['SCHG']==7), 'pstudent'] = 2 # university/professional school student
per_update.loc[pd.isnull(per_update['SCHG']), 'pstudent'] = 3 # non-student
per_update['pstudent'].value_counts()

3      458304
999     99661
1       36574
2       14291
Name: pstudent, dtype: int64

In [16]:
hh_update['hh_income_2018'] = 999
hh_update.loc[hh_update['ADJINC']==1080470, 'hh_income_2018'] = round((hh_update['HINCP']/1.0)*1.001264*1.07910576,2)
hh_update.loc[hh_update['ADJINC']==1073449, 'hh_income_2018'] = round((hh_update['HINCP']/1.0)*1.007588*1.06536503,2)
hh_update.loc[hh_update['ADJINC']==1054606, 'hh_income_2018'] = round((hh_update['HINCP']/1.0)*1.011189*1.04293629,2)
hh_update.loc[hh_update['ADJINC']==1031452, 'hh_income_2018'] = round((hh_update['HINCP']/1.0)*1.013097*1.01811790,2)
hh_update.loc[hh_update['ADJINC']==1010145, 'hh_income_2018'] = round((hh_update['HINCP']/1.0)*1.010145*1.00000000,2)

hh_update['hh_income_2017'] = hh_update['hh_income_2018'] * 0.97695
hh_update['hh_income_2010'] = hh_update['hh_income_2018'] * 0.87208
hh_update['hh_income_2000'] = hh_update['hh_income_2018'] * 0.68602

In [18]:
hh_update['hhinccat1'] = pd.cut(hh_update['hh_income_2000'], [-np.inf, 20000, 50000, 100000, np.inf], labels = [1, 2, 3, 4]).values.add_categories(999)
hh_update.loc[pd.isna(hh_update['HINCP']), 'hhinccat1'] = 999
hh_update['hhinccat1'].value_counts()

4      68723
3      65019
2      54216
1      34069
999    24290
Name: hhinccat1, dtype: int64